### Importamos las librerias necesarias para ejecutar nuestro modelo de analisis de sentimientos:

In [19]:
from pyspark.sql.functions import udf, when, col
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

### Descargamos el lexicon de Vader para nltk. 

* El lexicon de VADER es un conjunto de palabras y frases en inglés que han sido anotadas con puntajes de sentimiento. Cada palabra o frase en el lexicon tiene asignados valores de polaridad positiva, negativa y neutra, así como un puntaje compuesto que representa el sentimiento general asociado a esa palabra o frase.
* VADER (Valence Aware Dictionary and sEntiment Reasoner) es un analizador de sentimientos basado en reglas diseñado para el procesamiento del lenguaje natural. El lexicon de VADER es una de las componentes clave de este analizador.

In [20]:
nltk.download('vader_lexicon')

### Creamos una instancia del SentimentIntensityAnalyzer

In [21]:
sia = SentimentIntensityAnalyzer()

### Definimos una función UDF (User Defined Function) para aplicar el análisis de sentimiento y la convertimos a una funcion spark

In [22]:
def get_sentiment_score(text):
    return sia.polarity_scores(text)['compound']

In [23]:
# Convertir la función UDF en una función Spark
udf_get_sentiment_score = udf(get_sentiment_score, StringType())

### Cargamos los datos de `review-gold` para analizar las reseñas.

In [24]:
path_reviews_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/Yelpgold/reviews-gold/review-gold"
df_reviews = spark.read.format("parquet").load(path_reviews_gold)

In [25]:
df_reviews.show(5)

In [26]:
df_reviews.count()

### Aplicamos nuestro modelo.

In [27]:
# Calcular el puntaje de sentimiento compuesto y asignar la etiqueta de sentimiento
df_reviews = df_reviews.withColumn('sentiment_score', udf_get_sentiment_score(df_reviews['text']))
df_reviews = df_reviews.withColumn('sentiment', \
                                    when(col('sentiment_score') >= 0.3, 'positive') \
                                    .when((col('sentiment_score') > -0.3) & (col('sentiment_score') < 0.3), 'neutral') \
                                    .otherwise('negative'))

In [28]:
df_reviews.show(5)

### Creamos el df_sentiment con las columnas que nos interesan para ingestar en una tabla en la BBDD

In [29]:
df_sentiment = df_reviews.select("review_id", "sentiment_score", "sentiment")

### Cambiamos el tipo de dato de la columna 'sentiment_score' para evitar problemas al momento de la ingesta de los datos en la base de datos.

In [30]:
df_sentiment = df_sentiment.withColumn("sentiment_score", col("sentiment_score").cast("double"))

In [31]:
df_sentiment.dtypes

### Guardamos el DataFrame df_sentiment con las nuevas columnas en la tabla gold.

In [32]:
path_sentiment_gold = f"abfss://datumtech@datumlake.dfs.core.windows.net/gold/Yelpgold/sentiment-gold/review-sentiment"   
df_sentiment.write.format("parquet").save(path_sentiment_gold)